# Example: randomized complete block design (RCBD) with Sudoku

We again start by importing required modules

In [ ]:
using SudokuPlantDesign
using DataFrames
using XLSX
using PyPlot

It is worth noting the general idea of how to apply the Sudoku code to the RCBD before we proceed. In this type of design, there are only entry genotypes but not additional checks. However, the entries are replicated for different blocks and their relative arrangement among themselves has to be optimized.

To do so, we interprete the *checks* of the Sudoku design to be in fact the entries of the RCBD and *only* consider checks to be placed in a configuration such that there is only one check per block of each type. In this way, each block has exactly one genotype which is repeated across the different blocks.

## 1) Generate (optimized) Sudoku configuration

We start by generating a new configuration `conf` which is divided into 3 horizontal and 1 vertical block of dimensions `3` x `5` respectively. In total, there are `15` different genotypes to replicate which are now entering as checks into the configuration. They are initialized randomly with one check per type per block.

The configuration is plotted to show the initial starting point before optimization. 

In [ ]:
conf = get_configuration([3,3,3],[5],15, bc=:open)

initialize_checks_per_block!(conf)

show_configuration(conf, zoom=0.4)

mkpath("output_RCBD/")
savefig("output_RCBD/distribution_initial.pdf")

The cost function in this approach heavyly emphazises to only have one check per type per block but is left to be standard apart from this.

In [ ]:
function K_indiv(conf :: C) :: Float64 where {C <: CheckConfiguration}
    return  K_num_checks_equal_per_type(conf)*10 +
            K_checks_per_type_per_block(conf, 1)*20+
            K_neighbors_different_check_functional(conf, d->0.5/(d^3)) +
            K_neighbors_same_check_functional(conf, d->1/(d^3))
end

We only use a single update, namely the move to swap a check with another one. This preserves the original distribution of checks, namely that there is exactly one check per type.

In [ ]:
updates = [UpdateSwapCheckCheck()]

The optimization is run in the following

In [ ]:
costs = optimize_design!(
    conf,
    updates,
    K_indiv,
    500000
);

and the resulting configuration is visually checked by plotting

In [ ]:
show_configuration(conf, zoom=0.4)
mkpath("output_RCBD/")
savefig("output_RCBD/distribution_final.pdf")

## 2) Save design data with field plan

With an optimized configuration `conf` at hand, we can now proceed to create a field plan. For this, dataframes with the data for checks and entries are required.

In the case of our RCBD example, an Excel sheet with tabular input data `input_RCBD.xlsx` is provided. In contrast to the augmented design, it only containing a single sheet `genotypes`. This sheet is read in and converted into a dataframe to serve as the *checkdata*. The format of the sheet is analogous to the formot of the `input_augmented.xlsx` sheet, meaning it contains the name of the genotype in the first column, and in following columns further properties of the genotypes can be added to be transferred to the final output file and field plan.

Since in the RCBD, there are only checks and no entries, the entrydata (which is needed for the `fill_labels!` function later on) is simply an empty DataFrame.

In [ ]:
entrydata = DataFrame();

In [ ]:
checkdata = string.(DataFrame(XLSX.readtable("input_RCBD.xlsx", "genotypes")));
replace!.(eachcol(checkdata), "missing" => "NA");

To create a field plan, a labeled check configuration is created (based on the optimized configuration `conf` from the previous step). In this labeled configuration, both indices (the linear positions of the plants) as well as labels are set. The configuration is finally shown.

In [ ]:
lconf = LabeledCheckConfiguration(conf)

fill_indices_lines_y!(lconf, 1,-1, index_for_empty=false)
fill_labels!(lconf, checkdata, entrydata)

show_configuration(lconf, check_labels=false, zoom=0.4)
mkpath("output_RCBD/")
savefig("output_RCBD/final_sudoku_RCBD_design.pdf")

For exporting, the data of this optimized Sudoku-augmented design can now be converted back into a dataframe

In [ ]:
df = get_dataframe(lconf)

The dataframe can be modified in julia before exporting, here all generic property columns are renamed to the the column names of the checkdata dataframe.

In [ ]:
for (i,name) in enumerate(names(checkdata)[2:end])
    rename!(df,Symbol("property_"*string(i)) => Symbol(name))
end

df

Additional columns which hold the same data for all rows are inserted

In [ ]:
df[:, :year]       .= 2023
df[:, :extra_info] .= "myextrainfo"

df

finally, the dataframe is written back into an Excel file.

In [ ]:
mkpath("output_RCBD/")
XLSX.writetable("output_RCBD/final_sudoku_RCBD_design.xlsx", collect(eachcol(df)), names(df),overwrite=true)